In [1]:
import random
import time
import uuid

import names
import pymssql
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [2]:
#firstname, lastname, phone number, email
class fakeeduentry:

    def __init__(self):
        self.courses = ["PHYS210", "PHYS110", "CS231", "CS101", "MATH107", "MATH108"]
        self.evals = ["A", "B", "C", "D", "E", "F"]

    def generate_phonenumber(self):
        first = str(random.randint(100,999))
        second = str(random.randint(1,888)).zfill(3)
        last = (str(random.randint(1,9998)).zfill(4))
        while last in ['1111','2222','3333','4444','5555','6666','7777','8888']:
            last = (str(random.randint(1,9998)).zfill(4))
        return ('{}-{}-{}'.format(first,second, last))
    
    def generate_random_course(self):
        return self.courses[random.randint(0, len(self.courses)-1)]
    
    def generate_random_eval(self):
        return self.evals[random.randint(0, len(self.evals)-1)]


    #course, class
    def edu_2nd_entry_gen(self, uid):
        return (uid, names.get_first_name() + ' ' +  names.get_last_name(), self.generate_random_course())
    
    #eval, evaluation
    def edu_3rd_entry_gen(self, uid):
        return (uid, self.generate_random_course(), self.generate_random_eval())
    
    #registration, student
    def edu_4th_entry_gen(self, uid):
        return (uid, names.get_first_name() + ' ' +  names.get_last_name(), self.generate_phonenumber())
    
    def edu_all_entry_gen(self, uid):
        return (uid, names.get_first_name() + ' ' +  names.get_last_name(), self.generate_random_course(), self.generate_random_eval(), self.generate_phonenumber())

    def edu_all_db_gen(self, uid_beg, uid_end):
        print("generating fake data...")
        entries_all = []        
        for i in range(uid_beg, uid_end):
            entries_all.append(self.edu_all_entry_gen(i))            

        entries_2nd = [(x[0], x[1], x[2]) for x in entries_all]
        entries_3rd = [(x[0], x[2], x[3]) for x in entries_all]
        entries_4th = [(x[0], x[1], x[4]) for x in entries_all]
        return entries_2nd, entries_3rd, entries_4th
        
myfakeeduentry = fakeeduentry()
for i in range(100, 105):
    #thisedu2nd = myfakeeduentry.edu_2nd_entry_gen(i)
    #thisedu3rd = myfakeeduentry.edu_3rd_entry_gen(i)
    #thisedu4th = myfakeeduentry.edu_4th_entry_gen(i)
    #print(thisedu2nd)
    #print(thisedu3rd)
    #print(thisedu4th)
    thiseduall = myfakeeduentry.edu_all_entry_gen(i)
    print(thiseduall)

(100, 'Jennifer Brown', 'CS231', 'A', '784-194-2620')
(101, 'Oliver Carper', 'CS231', 'F', '985-775-6403')
(102, 'Erik Reeves', 'PHYS210', 'A', '127-446-1205')
(103, 'Michelle Scantling', 'CS231', 'E', '972-787-1254')
(104, 'Shirley Broughton', 'CS231', 'C', '370-197-7899')


In [8]:
class mssqldbprocess_edu:

    def __init__(self, hostip, usrname, pwd, dbnames):
        self.hostip = hostip
        self.usrname = usrname
        self.pwd = pwd
        self.dbnames = dbnames
    
    def createedu(self):
        for dbname in self.dbnames:
            conn = pymssql.connect(self.hostip, self.usrname, self.pwd, dbname)
            c1 = conn.cursor()
            # table 1 for uid
            if dbname == "course":
                c1.execute("IF OBJECT_ID('class', 'U') IS NOT NULL DROP TABLE class")
                c1.execute("CREATE TABLE class (ssn INT NOT NULL, name VARCHAR(100), course VARCHAR(100))")
            elif dbname == "eval":
                c1.execute("IF OBJECT_ID('evaluation', 'U') IS NOT NULL DROP TABLE evaluation")
                c1.execute("CREATE TABLE evaluation (ssn INT NOT NULL, course VARCHAR(100), eval VARCHAR(100))")
            elif dbname == "registration":
                c1.execute("IF OBJECT_ID('student', 'U') IS NOT NULL DROP TABLE student")
                c1.execute("CREATE TABLE student (ssn INT NOT NULL, name VARCHAR(100), phonenumber VARCHAR(100))")
            conn.commit()
            conn.close()
        return 
    
    def createhr(self):
        for dbname in self.dbnames:
            conn = pymssql.connect(self.hostip, self.usrname, self.pwd, dbname)
            c1 = conn.cursor()
            # table 1 for uid
            if dbname == "hr_pii":
                c1.execute("IF OBJECT_ID('employee', 'U') IS NOT NULL DROP TABLE employee")
                c1.execute("CREATE TABLE employee (eid INT NOT NULL, name VARCHAR(100), phone VARCHAR(100))")
            elif dbname == "hr_payment":
                c1.execute("IF OBJECT_ID('payment', 'U') IS NOT NULL DROP TABLE payment")
                c1.execute("CREATE TABLE payment (eid INT NOT NULL, payment VARCHAR(100), title VARCHAR(100))")
                c1.execute("IF OBJECT_ID('performance', 'U') IS NOT NULL DROP TABLE performance")
                c1.execute("CREATE TABLE performance (eid INT NOT NULL, bonus VARCHAR(100), attendence VARCHAR(100))")
            conn.commit()
            conn.close()
        return 
    
    def filledu(self, entries_2nd, entries_3rd, entries_4th):
        for dbname in self.dbnames:
            print("dumping " + dbname + "...")
            conn = pymssql.connect(self.hostip, self.usrname, self.pwd, dbname)
            c = conn.cursor()
            if dbname == "course":
                c.executemany("INSERT INTO class VALUES (%d, %s, %s)", entries_2nd)
            elif dbname == "eval":
                c.executemany("INSERT INTO evaluation VALUES (%d, %s, %s)", entries_3rd)
            elif dbname == "registration":
                c.executemany("INSERT INTO student VALUES (%d, %s, %s)", entries_4th)        
            conn.commit()
            conn.close()
        return

In [22]:
class mysqldbprocess_edu:

    def __init__(self, hostip, usrname, pwd, dbnames):
        self.hostip = hostip
        self.usrname = usrname
        self.pwd = pwd
        self.dbnames = dbnames
    
    def createedu(self):
        for dbname in self.dbnames:
            conn = pymysql.connect(self.hostip, self.usrname, self.pwd, dbname)
            c1 = conn.cursor()
            # table 1 for uid
            if dbname == "course":
                c1.execute("DROP TABLE IF EXISTS `class`;")
                c1.execute("CREATE TABLE class (ssn INT NOT NULL, name VARCHAR(100), course VARCHAR(100))")
            elif dbname == "eval":
                c1.execute("DROP TABLE IF EXISTS `evaluation`;")
                c1.execute("CREATE TABLE evaluation (ssn INT NOT NULL, course VARCHAR(100), eval VARCHAR(100))")
            elif dbname == "registration":
                c1.execute("DROP TABLE IF EXISTS `student`;")
                c1.execute("CREATE TABLE student (ssn INT NOT NULL, name VARCHAR(100), phonenumber VARCHAR(100))")
            conn.commit()
            conn.close()
        return 

    def createsales(self):
        for dbname in self.dbnames:
            conn = pymysql.connect(self.hostip, self.usrname, self.pwd, dbname)
            c1 = conn.cursor()
            # table 1 for uid
            if dbname == "sales_info":
                c1.execute("DROP TABLE IF EXISTS `sales`;")
                c1.execute("CREATE TABLE sales (eid INT NOT NULL, name VARCHAR(100), cids VARCHAR(100))")
            elif dbname == "customers_info":
                c1.execute("DROP TABLE IF EXISTS `customers`;")
                c1.execute("CREATE TABLE customers (cid INT NOT NULL, contact VARCHAR(100), trade VARCHAR(100))")
            conn.commit()
            conn.close()
        return 
    
    def filledu(self, entries_2nd, entries_3rd, entries_4th):
        for dbname in self.dbnames:
            print("dumping " + dbname + "...")
            conn = pymysql.connect(self.hostip, self.usrname, self.pwd, dbname)
            c = conn.cursor()
            if dbname == "course":
                c.executemany("INSERT INTO class VALUES (%s, %s, %s);", entries_2nd)
            elif dbname == "eval":
                c.executemany("INSERT INTO evaluation VALUES (%s, %s, %s)", entries_3rd)
            elif dbname == "registration":
                c.executemany("INSERT INTO student VALUES (%s, %s, %s)", entries_4th)
        
            conn.commit()
            conn.close()
        return

In [23]:
#mymssqldbprocess_edu = mssqldbprocess_edu("192.168.7.155", "SA", "Helios123", ["course", "eval", "registration"])
#mymssqldbprocess_edu.createedu()
#mymysqldbprocess_edu = mysqldbprocess_edu("192.168.7.74", "root", "Helios123", ["course", "eval", "registration"])
#mymysqldbprocess_edu.createedu()
#batch_size = 50
#for i in range(0, 2):
#    print("batch " + str(i))
#    my_entries_2nd, my_entries_3rd, my_entries_4th = myfakeeduentry.edu_all_db_gen(i*batch_size , (i+1)*batch_size)
#    mymssqldbprocess_edu.filledu(my_entries_2nd, my_entries_3rd, my_entries_4th)
#    mymysqldbprocess_edu.filledu(my_entries_2nd, my_entries_3rd, my_entries_4th)

In [25]:
mymssqldbprocess_hr = mssqldbprocess_edu("192.168.7.155", "SA", "Helios123", ["hr_pii", "hr_payment"])
mymssqldbprocess_hr.createhr()
mymysqldbprocess_sales = mysqldbprocess_edu("192.168.7.74", "root", "Helios123", ["sales_info", "customers_info"])
mymysqldbprocess_sales.createsales()

### End of the file